In [1]:
from PTO import random, solve

### Solving TSP with GRASP emulated in PTO

We'll use the same problem generator and fitness as when we solved TSP directly in PTO.

In [2]:
n = 10
def randprob(n): 
    c = [[0 for x in range(n)] for x in range(n)] # initialise cost matrix
    for i in range(n):
        for j in range(n):
            if i == j:
                c[i][j] = 0 # zero diagonal
            elif i > j:
                c[i][j] = c[j][i] # symmetric matrix
            else:
                c[i][j] = random.expovariate(1) # more interesting than uniform weights
    return c
instance = randprob(n)

In [3]:
def _fitness(perm, inst):
    # note negative indexing trick to include final step (return to origin)
    return -sum([inst[perm[i-1]][perm[i]] for i in range(0,len(perm))])
fitness = lambda perm: _fitness(perm, instance)

Next, we use the same generic GRASP generator as when we solved the ORDERING problem with GRASP emulated in PTO.

In [4]:
def randsol():
  solution = empty_solution()
  while(not complete(solution)):
    #print(solution)
    features = allowed_features(solution)
    costs = {feat:cost_feature(solution, feat) for feat in features}
    min_cost, max_cost = min(costs.values()), max(costs.values())
    RCL = [feat for feat in features if costs[feat] <= min_cost + alpha * (max_cost - min_cost)]
    #print(RCL)
    selected_feature = random.choice(RCL) # only source of randomness
    solution = add_feature(solution, selected_feature)
  return solution 

### TSP-specific functions for GRASP in PTO

Finally, we fill in the functions used by `randsol` in a way appropriate to TSP. In fact, all of them are the same as in the ORDERING problem, except for the `cost_feature` functions and a small bookkeeping change in `allowed_features` (we will now count from 0 to n-1 instead of 1 to n).

In [5]:
n=len(instance)

def empty_solution():
  return []

def complete(solution):
  return len(solution)==n

def allowed_features(solution):
  all_items = range(n) # count from 0 to n-1
  remaining_items = [item for item in all_items if item not in solution]
  return remaining_items

def cost_feature(solution, feat):
  if len(solution) == 0:
    # all cities cost nothing as start city. 
    # NB this will give a uniform random choice of start city,
    # which is better than hardcoding it!
    return 0 
  last_item = solution[-1]
  dist = instance[last_item][feat]
  return dist

def add_feature(solution, feat):
  sol = solution[:] + [feat]
  return sol

Now we can quickly test our approach without metaheuristic search. Again, we observe that the fully greedy approach is the best in this scenario.

In [6]:
alpha = 0.0 # completely greedy

for i in range(5):
    x = randsol()
    print("Random solution: fitness %.3f; %s" % (fitness(x), str(x)))
    
print("===")    
    
alpha = 0.5 # half way

for i in range(5):
    x = randsol()
    print("Random solution: fitness %.3f; %s" % (fitness(x), str(x)))
    
print("===")    
    
alpha = 1.0 # completely random

for i in range(5):
    x = randsol()
    print("Random solution: fitness %.3f; %s" % (fitness(x), str(x)))

Random solution: fitness -7.684; [1, 5, 3, 2, 9, 4, 7, 0, 8, 6]
Random solution: fitness -3.598; [2, 3, 6, 7, 9, 4, 1, 5, 8, 0]
Random solution: fitness -3.598; [2, 3, 6, 7, 9, 4, 1, 5, 8, 0]
Random solution: fitness -8.048; [3, 2, 9, 4, 7, 1, 5, 6, 8, 0]
Random solution: fitness -5.023; [0, 7, 9, 4, 1, 5, 3, 2, 6, 8]
===
Random solution: fitness -4.437; [6, 3, 4, 8, 9, 1, 5, 7, 0, 2]
Random solution: fitness -3.633; [3, 9, 8, 4, 1, 7, 0, 2, 6, 5]
Random solution: fitness -6.338; [9, 7, 2, 3, 8, 4, 1, 5, 6, 0]
Random solution: fitness -8.086; [9, 7, 5, 8, 4, 3, 2, 6, 0, 1]
Random solution: fitness -7.973; [0, 8, 5, 6, 4, 7, 1, 9, 2, 3]
===
Random solution: fitness -11.898; [2, 6, 4, 3, 8, 7, 0, 5, 9, 1]
Random solution: fitness -9.251; [1, 7, 5, 4, 0, 2, 3, 8, 6, 9]
Random solution: fitness -9.540; [9, 2, 4, 1, 8, 5, 3, 7, 0, 6]
Random solution: fitness -6.355; [8, 9, 4, 2, 1, 5, 6, 7, 0, 3]
Random solution: fitness -12.192; [2, 5, 1, 0, 6, 4, 9, 7, 8, 3]


Now we can test with metaheuristic optimisation. This time, we observe that a less greedy approach in the generator ($\alpha=0.5$) typically improves performance, at least with the HC solver.

In [7]:
print("alpha solver -fitness")
for alpha in [0, 0.5, 1.0]:
    for solver in ["RS", "HC", "EA", "MGA"]:
        ind, fit = solve(randsol, fitness, solver=solver, budget=150)
        print(alpha, solver, fit)


alpha solver -fitness
0 RS -3.598054431866717
0 HC -5.023215692594591
0 EA -3.598054431866717
IN MGA RUN
POP SIZE 12
[-5.023215692594592, -6.075571461961394, -6.075571461961394, -7.683583483829779, -5.023215692594592, -3.598054431866717, -5.023215692594591, -5.023215692594592, -5.68135922682716, -5.386446518387433, -6.075571461961394, -5.68135922682716]
0 MGA -3.598054431866717
0.5 RS -3.089009110342272
0.5 HC -3.22229523551451
0.5 EA -3.830096042186346
IN MGA RUN
POP SIZE 12
[-3.4344594617807767, -5.490652151914328, -3.2835460373747822, -8.273928680250176, -6.032596999039852, -4.829472751541084, -5.1857108916517545, -3.6285969633177757, -6.38283517708164, -8.105107102471981, -3.807218877744407, -6.020642171366156]
0.5 MGA -3.2835460373747822
1.0 RS -4.007868227643763
1.0 HC -4.913035098103312
1.0 EA -6.1188346756859495
IN MGA RUN
POP SIZE 12
[-10.965074902880552, -7.095758547832249, -10.325168650246399, -7.123802491784132, -13.276303827103243, -12.833392599446714, -11.68502209341539, 